In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

# 교차검증
- train의 정확도는 높은데 test 정확도는 낮으면 안됨
- 교차검증은 데이터의 편중을 막기 위해 별도의 여러 세트로 구성된 학습데이터와 검증데이터 세트로 학습과 평가를 수행하는 것

In [3]:
#데이터 불러오고 변수설정
fold_iris = load_iris()
features = fold_iris.data

label = fold_iris.target
fold_df_clf = DecisionTreeClassifier()

#### KFold란?
K개의 데이터 폴드를 만들어서 K번만큼 각 폴드 세트에 학습과 검증을 반복적으로 수행하는 방법

In [4]:
kfold = KFold(n_splits = 5)
cv_accuracy = [] #정확도
cv_precision = [] #정밀도
cv_recall = [] #재현율

In [5]:
kfold.split(features) #폴드별 학습용, 검증용 테스트의 row index를 array로 반환

<generator object _BaseKFold.split at 0x0000026FD68C8660>

In [9]:
n_iter = 0 #5번이니까 0부터 시작
for train_idx, test_idx in kfold.split(features):
    #반환된 인덴스를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    #학습 진행
    fold_df_clf.fit(X_train, y_train)
    #예측 진행
    fold_pred = fold_df_clf.predict(X_test)
    
    #반복시 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, fold_pred),5)
    print('\n 교차검증 정확도 : {:3f}'.format(n_iter, accuracy))
    cv_accuracy.append(accuracy)
    
print('\n')
print('\n 평균정확도', np.mean(accuracy))
    
    


 교차검증 정확도 : 1.000000

 교차검증 정확도 : 2.000000

 교차검증 정확도 : 3.000000

 교차검증 정확도 : 4.000000

 교차검증 정확도 : 5.000000



 평균정확도 0.73333


In [10]:
label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## KFold의 문제점

-iris 데이터는 타겟이 이진(0과1)이 아니라 0,1,2 세가지로 나뉜다. 
따라서 recall, precision 은 타겟값이 이진이 아니면 계산할 때 문제가 생길 수 있다

-KFold로 나눴을 때 0,1만 포함되거나 1,2만 포함되는 경우도 있어서 오류가 생길 수 있다. 

In [11]:
kfold_iris_data = load_iris()
kfold_iris_data_df = pd.DataFrame(data=kfold_iris_data.data, columns = kfold_iris_data.feature_names)
kfold_iris_data_df['target'] = kfold_iris_data.target
print(kfold_iris_data_df['target'].value_counts())

0    50
1    50
2    50
Name: target, dtype: int64


In [12]:
kfold_iris = KFold(n_splits = 3)
kfold_iris.split(kfold_iris_data_df)

<generator object _BaseKFold.split at 0x0000026FD69687B0>

In [13]:
kfold_iris_data_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [14]:
cnt_iter = 0
for train_idx, test_idx in kfold_iris.split(kfold_iris_data_df):
    cnt_iter += 1
    label_train = kfold_iris_data_df['target'].iloc[train_idx]
    label_test = kfold_iris_data_df['target'].iloc[test_idx]
    print('교차검증 :{}'.format(cnt_iter))
    print('학습 레이블데이터 분포\n', label_train.value_counts())
    print('검증 레이블데이터 분포\n', label_test.value_counts())

교차검증 :1
학습 레이블데이터 분포
 1    50
2    50
Name: target, dtype: int64
검증 레이블데이터 분포
 0    50
Name: target, dtype: int64
교차검증 :2
학습 레이블데이터 분포
 0    50
2    50
Name: target, dtype: int64
검증 레이블데이터 분포
 1    50
Name: target, dtype: int64
교차검증 :3
학습 레이블데이터 분포
 0    50
1    50
Name: target, dtype: int64
검증 레이블데이터 분포
 2    50
Name: target, dtype: int64


# StratifiedKFold
- 회귀 예측할때는 필요없음

In [15]:
from sklearn.model_selection import StratifiedKFold

In [18]:
skf_iris = StratifiedKFold(n_splits = 3)
cnt_iter = 0

In [19]:
cnt_iter = 0
for train_idx, test_idx in skf_iris.split(kfold_iris_data_df, kfold_iris_data_df['target']): #기존 데이터와 Y값도 같이 넣어야함
    cnt_iter += 1
    label_train = kfold_iris_data_df['target'].iloc[train_idx]
    label_test = kfold_iris_data_df['target'].iloc[test_idx]
    print('교차검증 :{}'.format(cnt_iter))
    print('학습 레이블데이터 분포\n', label_train.value_counts())
    print('검증 레이블데이터 분포\n', label_test.value_counts())

교차검증 :1
학습 레이블데이터 분포
 2    34
0    33
1    33
Name: target, dtype: int64
검증 레이블데이터 분포
 0    17
1    17
2    16
Name: target, dtype: int64
교차검증 :2
학습 레이블데이터 분포
 1    34
0    33
2    33
Name: target, dtype: int64
검증 레이블데이터 분포
 0    17
2    17
1    16
Name: target, dtype: int64
교차검증 :3
학습 레이블데이터 분포
 0    34
1    33
2    33
Name: target, dtype: int64
검증 레이블데이터 분포
 1    17
2    17
0    16
Name: target, dtype: int64


In [20]:
result_skfold = StratifiedKFold(n_splits=3)
result_clf = DecisionTreeClassifier(random_state=100)
n_iter = 0 # 5번 진행 예정이니 0 먼저 지정

cv_accuracy=[]
cv_precision=[]
cv_recall=[]


for train_idx, test_idx in result_skfold.split(features, label):
    X_train, X_test = features[train_idx], features[test_idx]
    
    y_train, y_test = label[train_idx], label[test_idx]
    
    #학습을 진행
    result_clf.fit(X_train, y_train)
    #예측
    pred = result_clf.predict(X_test)
    
    #정확도 관련해서 측정
    n_iter +=1 
    accuracy = np.round(accuracy_score(y_test, pred),3)
    precision = np.round(precision_score(y_test,pred, average='macro'),3)
    recall = np.round(recall_score(y_test, pred, average='macro'),3)
    print('\n{} 교차검증 정확도 : {}, 교차검증 precision :{}, 교차검증 recall :{}'.format(n_iter, accuracy, precision, recall))
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print('\n')
#print('\n 평균 정확도 ', np.mean(accuracy))
print('\n 평균 검증 정확도', np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))


1 교차검증 정확도 : 0.98, 교차검증 precision :0.981, 교차검증 recall :0.979

2 교차검증 정확도 : 0.92, 교차검증 precision :0.919, 교차검증 recall :0.919

3 교차검증 정확도 : 0.96, 교차검증 precision :0.961, 교차검증 recall :0.961



 평균 검증 정확도 0.9533333333333333 0.9536666666666666 0.953


정답이 3개라면 계산 식을 바꿔야한다. 그 정답 계산하는 식을 바꿔주는 인자가 average, micro, weighted


-micro : 전체의 평균
-macro : 평균 중의 평균
-weighted : 가중 평균